1. Интегрирование. Методы прямоугольников, трапеций, Симпсона. Реализовать вычисление по количеству разбиений отрезка (n) и по достигаемой точности (eps) по методу Рунге.

## Численное интегрирование

$\int_a^b f(x) dx \qquad f(x) \in [a,b]$

$\int_a^b f(x) dx = \sum_{k=1}^n A_k f(x_k)$ -- квадратурная формула $x_k \in [a,b]$

$A_k = \int_a^b l_k dx \qquad l_k$ -- коэффициент Лагранжа

$A_k = \int_a^b \frac {w(x)}{(x-x_k)w'(x_k)} dx$

$w(x) = \prod_{k=1}^n (x-x_k)$

$d$ -- целое натуральное, называется алгебраической степенью точности, если квадратурная формула точна для всех полиномов степени не выше $d$ и не точна для полиномов степени $d + 1$

### Квадратурные формулы прямоугольников.

#### Левых:

$\int_a^b f(x) dx = (b-a)f(a) \qquad d = 0$

#### Правых:

$\int_a^b f(x) dx = (b-a)f(b) \qquad d = 0$

#### Серединных:

$\int_a^b f(x) dx = (b-a)f(\frac{a+b}{2}) \qquad d = 1$

### Составная квадратурная формула.

$h = \frac {b-a}{N}, \, x_k= a+ k h, \, f_k=f(x_k), \, k = 0, \dots , N$

$\int_a^b f(x) dx = h \cdot \sum_{k=1}^{N} f(\delta + (k-1)h)$

$\delta = a $ -- левых прям.

$\delta = a+\frac{h}{2} $ -- средних.

$\delta = a + h $ -- правых прям.

### Квадратурная формула трапеций.

#### простая

$\int_a^b f(x) dx = \frac {b-a}{2} (f(a) + f(b)) \qquad d = 1$

#### составная

$\int_a^b f(x) dx = \frac {b-a}{2N} (f_0 + 2 (f_1 + \dots + f_{N-1}) + f_N)$

### Квадратурная формула Симпсона

#### простая

$\int_a^b f(x) dx = \frac {b-a}{6} (f(a) + 4f(\frac{a+b}{2}) + f(b)) \qquad d = 3 $

#### составная

$h=\frac{b-a}{2N} \qquad [x_{k-1}, x_{k+1}] \, k = 0, \dots , N$

$\int_a^b f(x) dx = \frac {b-a}{6N} (f_0 + 4(f_1 + f_3 + \cdots + f_{2N-1}) + 2(f_2 + f_4 + \cdots + f_{2N-2}) + f_{2N})$

### Оценка погрешности

$\exists f^{(d+1)}\in C[a,b]$

$| R_N (f) | = C(b-a)(\frac{b-a}{N})^{d+1} M_{d+1}$

$M_{d+1} = \max_{\xi \in [a,b]}|f^{d+1}(\xi)|$

$C = 1/2$ -- для левых и правых

$C = 1/24$ -- для средних

$C = 1/12$ -- для трапеций

$C = 1/2880$ -- для Симпсона


In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 61kB 27.4MB/s 
     |████████████████████████████████| 460kB 51.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/d5/351a6b47b20d417e82a669cf53f8cb4d7b55a57f73cbd05184
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Stored in directory: /root/.cache/pip/wheels/b3/7b/ae/5addd138cd8175503b9782737bada30d0c88310d08c106f9bf
Successfully built pycuda pytools mako


[1024 1024 64]

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

In [0]:
def kvadrature_GPU(a, b, N, delta):
  h = np.float32((b - a) /  N)
  delta = np.float32(delta)
  a = np.empty(N)
  a = a.astype(numpy.float32)
  a_gpu = cuda.mem_alloc(a.nbytes)
  cuda.memcpy_htod(a_gpu, a)
  
  mod = SourceModule("""
    __device__ float f(float x)
    {
      return x;
    }

    __global__ void kvadratur(float *a, float h, float delta)
    {
      int k = threadIdx.x;
      a[k] = f(delta + (k - 1) * h);
    }
    """)
  func = mod.get_function("kvadratur")
  func(a_gpu, h, delta, block=(N,1,1))
  a_res = numpy.empty_like(a)
  cuda.memcpy_dtoh(a_res, a_gpu)
  return h * sum(a_res)

In [0]:
def kvadrature_GPU_left(a, b, N):
  return kvadrature_GPU(a, b, N, a)

def kvadrature_GPU_midle(a, b, N):
  return kvadrature_GPU(a, b, N, a + (b - a) / (2 * N))

def kvadrature_GPU_right(a, b, N):
  return kvadrature_GPU(a, b, N, a + (b - a) / N)

In [27]:
kvadrature_GPU_left(-1, 1, 1024)

-0.005859375

In [28]:
kvadrature_GPU_midle(-1, 1, 1024)

-0.00390625

In [29]:
kvadrature_GPU_right(-1, 1, 1024)

-0.001953125